In [1]:
from abc import ABCMeta, abstractmethod
from functools import reduce


class Node(metaclass=ABCMeta):
    def __init__(self, name, parent=None):
        self.name = name
        self.parent = parent

    @property
    @abstractmethod
    def size(self):
        pass


class Dir(Node):
    def __init__(self, name, parent=None):
        super().__init__(name, parent)
        self.sub_dirs = {}
        self.files = {}

    @property
    def size(self):
        return reduce(
            lambda r, child_name: r + self.sub_dirs[child_name].size,
            self.sub_dirs.keys(),
            0,
        ) + reduce(
            lambda r, child_name: r + self.files[child_name].size, self.files.keys(), 0
        )


class File(Node):
    def __init__(self, name, parent=None, file_size=0):
        super().__init__(name, parent)
        self._file_size = file_size

    @property
    def size(self):
        return self._file_size

In [2]:
root = None
cur_dir = None
dir_list = []
with open("day_7_input.txt") as file:
    while line := file.readline().rstrip():
        if line == "$ cd /":
            if root is None:
                root = Dir("/")
                dir_list.append(root)
            cur_dir = root
        elif line == "$ cd ..":
            cur_dir = cur_dir.parent
        elif len(line) >= 4 and line[:4] == "$ cd":
            cur_dir = cur_dir.sub_dirs[line.split("cd ")[-1]]
        elif line == "$ ls":
            continue
        else:
            inp = line.split()
            new_name = inp[1]
            if inp[0] == "dir":
                cur_dir.sub_dirs[new_name] = Dir(new_name, cur_dir)
                dir_list.append(cur_dir.sub_dirs[new_name])
            else:
                cur_dir.files[new_name] = File(new_name, cur_dir, int(inp[0]))

In [3]:
sum([tree_dir.size for tree_dir in dir_list if tree_dir.size <= 100000])

1306611

In [4]:
root_size = root.size
min([tree_dir.size for tree_dir in dir_list if root_size - tree_dir.size <= 40000000])

13210366